- **Date:** 2019-6-2
- **Author:** Zhanyuan Zhang
- **Purpose:** 
    - Modify validation function and patch prediction function such that we can apply clipping on logits.
    - Test on subset (1,000/50,000) of ImageNet validation data set.

In [0]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/dl-security/') #Change the path to the directory that contains all code and data

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#!pip install https://github.com/bethgelab/foolbox/archive/master.zip

In [11]:
from bagnets.utils import plot_heatmap, generate_heatmap_pytorch
from bagnets.utils import pad_image, convert2channel_last, imagenet_preprocess, extract_patches, bagnet_predict, compare_heatmap
from bagnets.utils import bagnet33_debug, plot_saliency, compute_saliency_map
from bagnets.utils import get_topk_acc, validate
from bagnets.utils import bagnet_patch_predict, get_heatmap
from bagnets.clipping import*
from foolbox.utils import samples
import bagnets.pytorch
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
import time
import os
import cv2
img_path = "./ILSVRC2012_img_val"
root = "./"
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
if use_cuda:
    print(torch.cuda.get_device_name(0))

Tesla T4


In [5]:
# load pretrained model
bagnet33 = bagnets.pytorch.bagnet33(pretrained=True, avg_pool=False).to(device)
bagnet33.eval()
print()

In [0]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
imagenet_transform = transforms.Compose([transforms.Resize(256), 
                                          transforms.CenterCrop(224), 
                                          transforms.ToTensor(), 
                                          normalize])
imagenet_val = datasets.ImageNet('./', split='val', download=False, 
                                 transform=imagenet_transform)

In [0]:
val_subset_indices = np.random.choice(np.arange(50000), size=1000, replace=False)
val_subset_loader = torch.utils.data.DataLoader(imagenet_val, 
                                                batch_size=128,
                                                sampler=torch.utils.data.sampler.SubsetRandomSampler(val_subset_indices))

In [10]:
val_clip_pm_acc = validate(val_subset_loader, bagnet33, device, clip=clip_pm1)

Iteration 0, validation accuracy: 0.703, time: 0.0081634521484375s
Iteration 1, validation accuracy: 0.531, time: 0.008924245834350586s
Iteration 2, validation accuracy: 0.688, time: 0.008765459060668945s
Iteration 3, validation accuracy: 0.617, time: 0.008647680282592773s
Iteration 4, validation accuracy: 0.641, time: 0.008672952651977539s
Iteration 5, validation accuracy: 0.641, time: 0.00885629653930664s
Iteration 6, validation accuracy: 0.617, time: 0.0087890625s
Iteration 7, validation accuracy: 0.519, time: 0.009684324264526367s
Validation accuracy: 0.620, validation time: 0.07, total time: 22.32s


In [12]:
val_clip_bias_acc = validate(val_subset_loader, bagnet33, device, clip=clip_bias, b=10)

Iteration 0, validation accuracy: 0.430, time: 0.009350061416625977s
Iteration 1, validation accuracy: 0.398, time: 0.009202241897583008s
Iteration 2, validation accuracy: 0.422, time: 0.008630752563476562s
Iteration 3, validation accuracy: 0.375, time: 0.008682489395141602s
Iteration 4, validation accuracy: 0.383, time: 0.008728981018066406s
Iteration 5, validation accuracy: 0.430, time: 0.009140253067016602s
Iteration 6, validation accuracy: 0.422, time: 0.008927583694458008s
Iteration 7, validation accuracy: 0.365, time: 0.009010076522827148s
Validation accuracy: 0.403, validation time: 0.07, total time: 22.46s


In [13]:
val_clip_linear_acc = validate(val_subset_loader, bagnet33, device, clip=clip_linear, a=0.005, b=0.01)

Iteration 0, validation accuracy: 0.852, time: 0.009200572967529297s
Iteration 1, validation accuracy: 0.875, time: 0.008899688720703125s
Iteration 2, validation accuracy: 0.820, time: 0.009708404541015625s
Iteration 3, validation accuracy: 0.867, time: 0.009101629257202148s
Iteration 4, validation accuracy: 0.883, time: 0.00879812240600586s
Iteration 5, validation accuracy: 0.820, time: 0.008843660354614258s
Iteration 6, validation accuracy: 0.859, time: 0.008672952651977539s
Iteration 7, validation accuracy: 0.817, time: 0.009046792984008789s
Validation accuracy: 0.849, validation time: 0.07, total time: 22.81s


In [14]:
val_sigmoid_linear_acc = validate(val_subset_loader, bagnet33, device, clip=sigmoid_linear, a=0.005, b=0.01)

Iteration 0, validation accuracy: 0.859, time: 0.008400678634643555s
Iteration 1, validation accuracy: 0.844, time: 0.00866842269897461s
Iteration 2, validation accuracy: 0.828, time: 0.008819818496704102s
Iteration 3, validation accuracy: 0.906, time: 0.008807659149169922s
Iteration 4, validation accuracy: 0.852, time: 0.008734941482543945s
Iteration 5, validation accuracy: 0.820, time: 0.008758544921875s
Iteration 6, validation accuracy: 0.852, time: 0.008992671966552734s
Iteration 7, validation accuracy: 0.846, time: 0.00857234001159668s
Validation accuracy: 0.851, validation time: 0.07, total time: 22.70s


In [15]:
val_tanh_linear_acc = validate(val_subset_loader, bagnet33, device, clip=tanh_linear, a=0.005, b=0.01)

Iteration 0, validation accuracy: 0.867, time: 0.008697986602783203s
Iteration 1, validation accuracy: 0.836, time: 0.008907794952392578s
Iteration 2, validation accuracy: 0.898, time: 0.008799076080322266s
Iteration 3, validation accuracy: 0.883, time: 0.008912086486816406s
Iteration 4, validation accuracy: 0.797, time: 0.009010076522827148s
Iteration 5, validation accuracy: 0.805, time: 0.009046077728271484s
Iteration 6, validation accuracy: 0.828, time: 0.008885860443115234s
Iteration 7, validation accuracy: 0.894, time: 0.008668184280395508s
Validation accuracy: 0.851, validation time: 0.07, total time: 22.67s
